In [1]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [2]:
# conn_aws = Utilities.get_athena_dev_aws_connection()
conn_aws = Utilities.get_athena_qa_aws_connection()

In [3]:
sql = """
SELECT * FROM meter_events.event_summ_regex_setup
"""


In [4]:
regex_setup_df = pd.read_sql(sql, conn_aws, dtype=str)

C:\Temp/ipykernel_2992/1022206307.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  regex_setup_df = pd.read_sql(sql, conn_aws, dtype=str)


In [5]:
regex_setup_df

,regex_id,enddeviceeventtypeid,regex_seq_no,regex_search_pattern,regex_replacement_pattern,pivot_group,pivot_id,printing_ord_nb,regex_status,regex_descr,regex_report_title,regex_category,itron_event_id,itron_event_desc,eff_dt,aws_update_dttm
0,r1,3.21.38.248,1,(Detected end of voltage swell)(?:(?:\s*(?:for...,$1,1,cr1,1,A,,,,,,2023-04-11,2023-04-11 08:30:04
1,r2,3.21.38.248,2,(Detected end of voltage swell)(?:(?:\s*(?:for...,$1,1,cr2,2,A,,Detected end of voltage swell,,,,2023-04-11,2023-04-11 08:30:04
2,r3,3.21.38.223,1,(Detected end of voltage sag)(?:(?:\s*(?:for|o...,$1,2,cr3,3,A,,Detected end of voltage sag,,,,2023-04-11,2023-04-11 08:30:04
3,r4,3.21.38.223,2,(Detected end of voltage sag)(?:(?:\s*(?:for|o...,$1,2,cr4,4,A,,,,,,2023-04-11,2023-04-11 08:30:04
4,r5,3.12.136.85,1,(.*) failed consecutively for [0-9]+ times?(?:...,$1 failed consecutively for 1 or more times,3,cr5,5,A,,NET_MGMT command failed consecutively for 1 or...,,,,2023-04-11,2023-04-11 08:30:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,r129,3.22.19.242,1,(Meter event Test Mode Started)\s*Time event o...,$1,73,cr211,211,A,,,,,,2023-04-11,2023-04-11 08:30:04
211,r130,3.22.19.242,2,(Meter event Test Mode Started).*,$1,73,cr212,212,A,,,,,,2023-04-11,2023-04-11 08:30:04
212,r131,3.33.1.257,1,(Meter event Tamper Attempt Suspected)\s*Time ...,$1,74,cr213,213,A,,,,,,2023-04-11,2023-04-11 08:30:04
213,r132,3.33.1.257,2,(Meter event Tamper Attempt Suspected).*,$1,74,cr214,214,A,,,,,,2023-04-11,2023-04-11 08:30:04


In [6]:
regex_setup_df[['pivot_id', 'regex_report_title']]

,pivot_id,regex_report_title
0,cr1,
1,cr2,Detected end of voltage swell
2,cr3,Detected end of voltage sag
3,cr4,
4,cr5,NET_MGMT command failed consecutively for 1 or...
...,...,...
210,cr211,
211,cr212,
212,cr213,
213,cr214,


In [7]:
regex_setup_df[regex_setup_df['regex_report_title']=='Primary Power Up']

,regex_id,enddeviceeventtypeid,regex_seq_no,regex_search_pattern,regex_replacement_pattern,pivot_group,pivot_id,printing_ord_nb,regex_status,regex_descr,regex_report_title,regex_category,itron_event_id,itron_event_desc,eff_dt,aws_update_dttm
14,r13,3.26.0.216,1,(Primary Power Up)(?:(?:\s*occurred\s*)?(?:\s*...,$1,6,cr15,15,A,,Primary Power Up,,,,2023-04-11,2023-04-11 08:30:04
15,r14,3.26.0.216,2,Meter event (Primary Power Up)\s*Time event oc...,$1,6,cr16,16,A,,Primary Power Up,,,,2023-04-11,2023-04-11 08:30:04


In [8]:
(regex_setup_df['regex_report_title']=='').sum()

49

In [9]:
regex_setup_df['regex_report_title'].value_counts()

regex_report_title
                                                                     49
Primary Power Up                                                      2
Meter event Reverse Rotation Detected                                 2
Meter event Demand Reset Occurred                                     2
Meter event Measurement Error Detected                                2
                                                                     ..
Under Voltage (CA000400) Phase  C Voltage out of tolerance.           1
Under Voltage (CA000400) Phase  A, B, C Voltage out of tolerance.     1
Under Voltage (CA000400) Phase  A Voltage out of tolerance.           1
Under Voltage (CA000400) Phase  B, C Voltage out of tolerance.        1
Meter event Clock Error Detected                                      1
Name: count, Length: 160, dtype: int64

In [10]:
dups = regex_setup_df['regex_report_title'].value_counts()

In [11]:
dups[dups==2].index.tolist()

['Primary Power Up',
 'Meter event Reverse Rotation Detected',
 'Meter event Demand Reset Occurred',
 'Meter event Measurement Error Detected',
 'Meter event Nonvolatile Memory Failure Detected',
 'Detected end of voltage swell',
 'Detected end of voltage sag']

In [12]:
regex_setup_df[regex_setup_df['regex_report_title'].isin(dups[dups==2].index)].sort_values(by=['regex_report_title'])

,regex_id,enddeviceeventtypeid,regex_seq_no,regex_search_pattern,regex_replacement_pattern,pivot_group,pivot_id,printing_ord_nb,regex_status,regex_descr,regex_report_title,regex_category,itron_event_id,itron_event_desc,eff_dt,aws_update_dttm
2,r3,3.21.38.223,1,(Detected end of voltage sag)(?:(?:\s*(?:for|o...,$1,2,cr3,3,A,,Detected end of voltage sag,,,,2023-04-11,2023-04-11 08:30:04
134,r56,3.21.43.223,1,(Detected end of voltage sag)(?:(?:\s*(?:for|o...,$1,31,cr135,135,A,,Detected end of voltage sag,,,,2023-04-11,2023-04-11 08:30:04
1,r2,3.21.38.248,2,(Detected end of voltage swell)(?:(?:\s*(?:for...,$1,1,cr2,2,A,,Detected end of voltage swell,,,,2023-04-11,2023-04-11 08:30:04
137,r59,3.21.43.248,2,(Detected end of voltage swell)(?:(?:\s*(?:for...,$1,32,cr138,138,A,,Detected end of voltage swell,,,,2023-04-11,2023-04-11 08:30:04
128,r52,3.12.93.219,2,(Meter event (?:Reverse Rotation Detected|Dema...,$1,29,cr129,129,A,,Meter event Demand Reset Occurred,,,,2023-04-11,2023-04-11 08:30:04
164,r84,3.18.72.85,2,(Meter event (?:Nonvolatile Memory Failure Det...,$1,50,cr165,165,A,,Meter event Demand Reset Occurred,,,,2023-04-11,2023-04-11 08:30:04
144,r64,3.21.67.79,2,(Meter event Measurement Error Detected)\s*Tim...,$1,36,cr145,145,A,,Meter event Measurement Error Detected,,,,2023-04-11,2023-04-11 08:30:04
146,r66,3.21.67.79,4,(Meter detected a measurement error).*,Meter event Measurement Error Detected,36,cr147,147,A,,Meter event Measurement Error Detected,,,,2023-04-11,2023-04-11 08:30:04
163,r83,3.18.72.85,1,(Meter event.*?)\s*Time event occurred on mete...,$1,50,cr164,164,A,,Meter event Nonvolatile Memory Failure Detected,,,,2023-04-11,2023-04-11 08:30:04
166,r85,3.18.72.85,3,(Meter detected a nonvolatile memory failure).*,Meter event Nonvolatile Memory Failure Detected,50,cr167,167,A,,Meter event Nonvolatile Memory Failure Detected,,,,2023-04-11,2023-04-11 08:30:04


In [13]:
regex_setup_df[regex_setup_df['pivot_id']=='cr151']

,regex_id,enddeviceeventtypeid,regex_seq_no,regex_search_pattern,regex_replacement_pattern,pivot_group,pivot_id,printing_ord_nb,regex_status,regex_descr,regex_report_title,regex_category,itron_event_id,itron_event_desc,eff_dt,aws_update_dttm
150,r70,3.35.0.40,3,(Meter event AX Temp Threshold Exceeded)\s*Tim...,$1,38,cr151,151,A,,,,,,2023-04-11,2023-04-11 08:30:04


In [14]:
natsorted(regex_setup_df['regex_report_title'].unique().tolist())

['',
 'Access Point has lost connectivity with FHSS 900 MHz band.',
 'Cleared: Meter cleared tamper detection (C1219 Table 3)',
 'Cleared: Meter detected a high temperature condition. (C1219 Table 3)',
 'DSP Error (C1219 Table 3: Er200000)',
 'DSP Error cleared',
 'Demand Reset',
 'Detected end of voltage sag',
 'Detected end of voltage swell',
 'Device Failed: Reason: Security public key mismatch',
 'Device Failed: Reason: unknown (0x4)',
 'Device Failed: Reason: unknown (0x6)',
 'Device exceeded the max allowable trap threshold',
 'Diag1 Condition cleared',
 'Diag1: Polarity, Cross Phase, Reverse Energy Flow: Angle out of tolerance.',
 'Diag6 Condition cleared',
 'Diag6: Under Voltage, Element A',
 'Diag7 Condition cleared',
 'Diag7: Over Voltage, Element A',
 'Error occurred when attempting to synch meter time with NIC time for device',
 'Ignoring Interval Read data for device as it has time in the future',
 'Ignoring Register Read data for device as it has time in the future',
 'KV

In [15]:
regex_setup_df[regex_setup_df['regex_report_title'].str.contains('Nonvolatile')]

,regex_id,enddeviceeventtypeid,regex_seq_no,regex_search_pattern,regex_replacement_pattern,pivot_group,pivot_id,printing_ord_nb,regex_status,regex_descr,regex_report_title,regex_category,itron_event_id,itron_event_desc,eff_dt,aws_update_dttm
163,r83,3.18.72.85,1,(Meter event.*?)\s*Time event occurred on mete...,$1,50,cr164,164,A,,Meter event Nonvolatile Memory Failure Detected,,,,2023-04-11,2023-04-11 08:30:04
166,r85,3.18.72.85,3,(Meter detected a nonvolatile memory failure).*,Meter event Nonvolatile Memory Failure Detected,50,cr167,167,A,,Meter event Nonvolatile Memory Failure Detected,,,,2023-04-11,2023-04-11 08:30:04


In [16]:
strng = 'Service disconnect operation occurred with status: Close operation succeeded '
pattern = r'(Service disconnect operation occurred with status:) (Open|Close) operation (started|succeeded)\s*'
repl=r'\3'
re.sub(pattern, repl, strng)

'succeeded'

In [17]:
regex_setup_df[regex_setup_df['regex_report_title']=='Meter event Nonvolatile Memory Failure Detected  ']

,regex_id,enddeviceeventtypeid,regex_seq_no,regex_search_pattern,regex_replacement_pattern,pivot_group,pivot_id,printing_ord_nb,regex_status,regex_descr,regex_report_title,regex_category,itron_event_id,itron_event_desc,eff_dt,aws_update_dttm


In [18]:
save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\events_summary_vw'
date_ranges = [
    ['2023-04-01', '2023-04-07'], 
    ['2023-04-08', '2023-04-14'], 
    ['2023-04-15', '2023-04-21'], 
    ['2023-04-22', '2023-04-28'], 
    ['2023-04-29', '2023-05-05'], 
    ['2023-05-06', '2023-05-12'], 
    ['2023-05-13', '2023-05-19'], 
    ['2023-05-20', '2023-05-26'], 
    ['2023-05-27', '2023-05-31'], 
]
opcos = [
#     'ap', 
#     'oh', 
#     'im', 
#     'pso', 
#     'swp', 
    'tx'
]

In [19]:
for opco in opcos:
    for date_range in date_ranges:
        end_events_sql_function_kwargs=dict(
            schema_name='meter_events', 
            table_name='events_summary_vw', 
            cols_of_interest=['*'], 
            date_range=date_range, 
           opco=opco
        )
        save_name = f"end_events_{opco}_{date_range[0].replace('-','')}_{date_range[1].replace('-','')}.pkl"
        #-----
        end_events = AMIEndEvents(
            df_construct_type=DFConstructType.kRunSqlQuery, 
            contstruct_df_args = dict(conn_db=conn_aws), 
            build_sql_function=AMIEndEvents_SQL.build_sql_end_events, 
            build_sql_function_kwargs=end_events_sql_function_kwargs, 
            init_df_in_constructor=True, 
            save_args=False
        )
        end_events.df.to_pickle(os.path.join(save_dir, save_name))

C:\Users\s346557\Documents\Analysis\GenAn.py:617: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn_db, **read_sql_args)
C:\Users\s346557\Documents\Analysis\GenAn.py:617: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn_db, **read_sql_args)
C:\Users\s346557\Documents\Analysis\GenAn.py:617: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn_db, **read_sql_args)
C:\Users\s346557\Documents\Analysis\GenAn.py:617: UserWarning: pandas only supp

In [20]:
assert(0)

AssertionError: 

In [ ]:
ede_df_1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\events_summary_vw\end_events_ap_20230401_20230407.pkl')
ede_df_2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\RCPO_Quick\end_events_ap_20230401_20230407.pkl')

In [ ]:
rename_cols_dict = {x[0]:x[1] for x in regex_setup_df[['pivot_id', 'regex_report_title']].values.tolist()}
ede_df_1=ede_df_1.rename(columns=rename_cols_dict)

In [ ]:
# Make sure other columns found in ede_df_1 are all zero
ede_df_1_notin2 = ede_df_1[list(set(ede_df_1.columns.tolist()).difference(set(ede_df_2.columns.tolist())))]
cols_to_ignore_in_sum = ['trsf_pole_nb', 'xf_meter_cnt', 'events_tot', 'aep_opco']
ede_df_1_notin2 = ede_df_1_notin2.drop(columns=cols_to_ignore_in_sum)
assert(ede_df_1_notin2.sum().sum()==0)

In [ ]:
ede_df_1_in2 = ede_df_1[ede_df_2.columns.tolist()].copy()
start=time.time()
# Combine degenerate columns
col_counts = ede_df_1_in2.columns.value_counts()
for mult_col in col_counts[col_counts>1].index.tolist():
    agg_col = ede_df_1_in2[mult_col].sum(axis=1)
    ede_df_1_in2 = ede_df_1_in2.drop(columns=mult_col)
    ede_df_1_in2[mult_col] = agg_col
print(time.time()-start)

In [ ]:
ede_df_2.shape

In [ ]:
ede_df_1_in2.shape

In [ ]:
assert(ede_df_1_in2.shape[1]==ede_df_2.shape[1])
assert(len(set(ede_df_1_in2.columns.tolist()).symmetric_difference(set(ede_df_2.columns.tolist())))==0)
ede_df_1_in2 = ede_df_1_in2[ede_df_2.columns]
#-----
ede_df_1_in2 = ede_df_1_in2.sort_values(by=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], ignore_index=True)
ede_df_2     = ede_df_2.sort_values(by=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], ignore_index=True)

In [ ]:
ede_df_1_in2.equals(ede_df_2)

In [ ]:
dffs = Utilities_df.get_dfs_diff(ede_df_1_in2, ede_df_2)
dffs

In [ ]:
rename_cols_dict

In [ ]:
ede_df_1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\events_summary_vw\end_events_ap_20230401_20230407.pkl')
ede_df_2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\Case_Sensitive\RCPO_Quick\end_events_ap_20230401_20230407.pkl')

In [ ]:
cols_to_ignore = ['trsf_pole_nb', 'xf_meter_cnt', 'events_tot', 'aep_opco']
ede_df_1 = ede_df_1.drop(columns=cols_to_ignore)
#-----
rename_cols_dict = {x[0]:x[1] for x in regex_setup_df[['pivot_id', 'regex_report_title']].values.tolist()}
ede_df_1=ede_df_1.rename(columns=rename_cols_dict)

In [ ]:
ede_df_1.columns.value_counts()

In [ ]:
ede_df_1

In [ ]:
ede_df_1.drop(columns=[''])

In [ ]:
cols_to_ignore = ['trsf_pole_nb', 'xf_meter_cnt', 'events_tot', 'aep_opco']
ede_df_1 = ede_df_1.drop(columns=cols_to_ignore)
#-----
rename_cols_dict = {x[0]:x[1] for x in regex_setup_df[['pivot_id', 'regex_report_title']].values.tolist()}
ede_df_1=ede_df_1.rename(columns=rename_cols_dict)
# Combine degenerate columns
col_counts = ede_df_1.columns.value_counts()
for mult_col in col_counts[col_counts>1].index.tolist():
    agg_col = ede_df_1[mult_col].sum(axis=1)
    ede_df_1 = ede_df_1.drop(columns=mult_col)
    ede_df_1[mult_col] = agg_col
#-----
print('Before make columns eq')
print(ede_df_1.shape)
print(ede_df_2.shape)
#-----
AMIEndEvents.make_reason_counts_per_outg_columns_equal(
    rcpo_df_1=ede_df_1, 
    rcpo_df_2=ede_df_2, 
    same_order=True, 
    inplace=True, 
    cols_to_init_with_empty_lists=None
)
#-----
print('After make columns eq')
print(ede_df_1.shape)
print(ede_df_2.shape)
#-----
ede_df_1 = ede_df_1.sort_values(by=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], ignore_index=True)
ede_df_2 = ede_df_2.sort_values(by=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], ignore_index=True)
#-----

In [ ]:
ede_df_1.equals(ede_df_2)

In [ ]:
dffs = Utilities_df.get_dfs_diff(ede_df_1, ede_df_2)
dffs

In [ ]:
dffs.head(10)

In [ ]:
ede_df_1[['Meter event S4TemperatureThreshold', 'NIC backup battery inactive', 'NIC Power Restore Trap Received from device', 'serialnumber', 'aep_premise_nb', 'aep_event_dt']].head()

In [ ]:
ede_df_2[['Meter event S4TemperatureThreshold', 'NIC backup battery inactive', 'NIC Power Restore Trap Received from device', 'serialnumber', 'aep_premise_nb', 'aep_event_dt']].head()

In [ ]:
ede_df_1.drop(columns=['serialnumber', 'aep_premise_nb', 'aep_event_dt']).iloc[0].sum()

In [ ]:
ede_df_2.drop(columns=['serialnumber', 'aep_premise_nb', 'aep_event_dt']).iloc[0].sum()

In [ ]:
cr154 cr9

In [ ]:
regex_setup_df[regex_setup_df['pivot_id']=='cr154']

In [ ]:
regex_setup_df[regex_setup_df['pivot_id']=='cr9']

In [ ]:
regex_setup_df[regex_setup_df['regex_report_title']=='Meter event S4TemperatureThreshold']

In [ ]:
ede_df_1.columns.tolist()

In [ ]:
ede_df_1.columns.tolist()

In [ ]:
list(set(ede_df_2.columns.tolist()).difference(set(ede_df_1.columns.tolist())))

In [ ]:
list(set(ede_df_1.columns.tolist()).difference(set(ede_df_2.columns.tolist())))

In [22]:
save_dir_1 = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\events_summary_vw'
save_dir_2 = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\Case_Sensitive\RCPO_Quick'
# save_dir_1 = r'U:\CloudData\dovs_and_end_events_data\regex_check\events_summary_vw'
# save_dir_2 = r'U:\CloudData\dovs_and_end_events_data\regex_check\Case_Sensitive\RCPO_Quick'
diffs_dict = {}

for opco in opcos:
    for date_range in date_ranges:
        save_name = f"end_events_{opco}_{date_range[0].replace('-','')}_{date_range[1].replace('-','')}.pkl"
        #-----
        ede_df_1 = pd.read_pickle(os.path.join(save_dir_1, save_name))
        ede_df_2 = pd.read_pickle(os.path.join(save_dir_2, save_name))
        #-----
        cols_to_ignore = ['trsf_pole_nb', 'xf_meter_cnt', 'events_tot', 'aep_opco']
        ede_df_1 = ede_df_1.drop(columns=cols_to_ignore)
        #-----
        rename_cols_dict = {x[0]:x[1] for x in regex_setup_df[['pivot_id', 'regex_report_title']].values.tolist()}
        ede_df_1=ede_df_1.rename(columns=rename_cols_dict)
        # Combine degenerate columns
        col_counts = ede_df_1.columns.value_counts()
        for mult_col in col_counts[col_counts>1].index.tolist():
            agg_col = ede_df_1[mult_col].sum(axis=1)
            ede_df_1 = ede_df_1.drop(columns=mult_col)
            ede_df_1[mult_col] = agg_col
        #-----
        cols_in_2_not_1 = list(set(ede_df_2.columns.tolist()).difference(set(ede_df_1.columns.tolist())))
        #-----
        AMIEndEvents.make_reason_counts_per_outg_columns_equal(
            rcpo_df_1=ede_df_1, 
            rcpo_df_2=ede_df_2, 
            same_order=True, 
            inplace=True, 
            cols_to_init_with_empty_lists=None
        )
        #-----
        ede_df_1 = ede_df_1.sort_values(by=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], ignore_index=True)
        ede_df_2 = ede_df_2.sort_values(by=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], ignore_index=True)
        #-----
        diffs_df = Utilities_df.get_dfs_diff(ede_df_1, ede_df_2)
        #-----
        assert(save_name not in diffs_dict.keys())
        diffs_dict[save_name] = dict(
            cols_in_2_not_1=cols_in_2_not_1, 
            diffs_df=diffs_df
        )
        #-----
        if ede_df_1.equals(ede_df_2):
            continue
        else:
            diffs_df = Utilities_df.get_dfs_diff(ede_df_1, ede_df_2)
            if diffs_df.shape[0]==0:
                continue
            else:
                print(f"NOT EQUAL: {save_name}")

C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

...

C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rcpo_df_2[col] = [0.0]*n_rows_2
C:\Users\s346557\Documents\Analysis\AMIEndEvents.py:2431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

In [23]:
with open(os.path.join(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\Case_Sensitive', 'diffs_dict.pkl'), 'wb') as handle:
    pickle.dump(diffs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(os.path.join(r'U:\CloudData\dovs_and_end_events_data\regex_check\Case_Sensitive', 'diffs_dict.pkl'), 'wb') as handle:
#     pickle.dump(diffs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open(os.path.join(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\Case_Sensitive', 'diffs_dict.pkl'), 'rb') as handle:
#     diffs_dict = pickle.load(handle)

In [ ]:
assert(0)

In [24]:
diffs_dict

{'end_events_tx_20230401_20230407.pkl': {'cols_in_2_not_1': [],
  'diffs_df': Empty DataFrame
  Columns: [df1_values, df2_values]
  Index: []},
 'end_events_tx_20230408_20230414.pkl': {'cols_in_2_not_1': [],
  'diffs_df': Empty DataFrame
  Columns: [df1_values, df2_values]
  Index: []},
 'end_events_tx_20230415_20230421.pkl': {'cols_in_2_not_1': [],
  'diffs_df': Empty DataFrame
  Columns: [df1_values, df2_values]
  Index: []},
 'end_events_tx_20230422_20230428.pkl': {'cols_in_2_not_1': [],
  'diffs_df': Empty DataFrame
  Columns: [df1_values, df2_values]
  Index: []},
 'end_events_tx_20230429_20230505.pkl': {'cols_in_2_not_1': [],
  'diffs_df': Empty DataFrame
  Columns: [df1_values, df2_values]
  Index: []},
 'end_events_tx_20230506_20230512.pkl': {'cols_in_2_not_1': [],
  'diffs_df': Empty DataFrame
  Columns: [df1_values, df2_values]
  Index: []},
 'end_events_tx_20230513_20230519.pkl': {'cols_in_2_not_1': [],
  'diffs_df': Empty DataFrame
  Columns: [df1_values, df2_values]
  Inde

In [25]:
ede_df_2

,,Access Point has lost connectivity with FHSS 900 MHz band.,Cleared: Meter cleared tamper detection (C1219 Table 3),Cleared: Meter detected a high temperature condition. (C1219 Table 3),DSP Error (C1219 Table 3: Er200000),DSP Error cleared,Demand Reset,Detected end of voltage sag,Detected end of voltage swell,Device Failed: Reason: Security public key mismatch,...,Under Voltage (CA000400) Phase A and C.,Under Voltage (CA000400) Phase A.,Under Voltage (CA000400) Phase C.,Under Voltage (CA000400) cleared,Under Voltage (Diagnostic 6) Phase A.,Under Voltage (Diagnostic 6) cleared,Universal event Unsupported Event(41) with priority Alarm,aep_event_dt,aep_premise_nb,serialnumber
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-28,949590902,104360593
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-29,949590902,104360593
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-31,941560772,104360746
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-31,946502705,106181925
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-31,948388684,106191379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-31,942444514,592811790
109596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-27,944750273,592811807
109597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-28,944750273,592811807
109598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-30,944750273,592811807


# NEW

In [ ]:
save_dir_1 = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\events_summary_vw'
save_dir_2 = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\Case_Sensitive\RCPO_Quick'
# save_dir_1 = r'U:\CloudData\dovs_and_end_events_data\regex_check\events_summary_vw'
# save_dir_2 = r'U:\CloudData\dovs_and_end_events_data\regex_check\Case_Sensitive\RCPO_Quick'
diffs_dict = {}

for opco in opcos:
    for date_range in date_ranges:
        save_name = f"end_events_{opco}_{date_range[0].replace('-','')}_{date_range[1].replace('-','')}.pkl"
        #-----
        ede_df_1 = pd.read_pickle(os.path.join(save_dir_1, save_name))
        ede_df_2 = pd.read_pickle(os.path.join(save_dir_2, save_name))
        #-----
        cols_to_ignore = ['trsf_pole_nb', 'xf_meter_cnt', 'events_tot', 'aep_opco']
        ede_df_1 = ede_df_1.drop(columns=cols_to_ignore)
        #-----
        rename_cols_dict = {x[0]:x[1] for x in regex_setup_df[['pivot_id', 'regex_report_title']].values.tolist()}
        ede_df_1=ede_df_1.rename(columns=rename_cols_dict)
        # Drop empty columns
        ede_df_1 = ede_df_1.drop(columns=[''])
        col_counts = ede_df_1.columns.value_counts()
        assert((col_counts==1).all())
        #-----
        cols_in_2_not_1 = list(set(ede_df_2.columns.tolist()).difference(set(ede_df_1.columns.tolist())))
        #-----
        AMIEndEvents.make_reason_counts_per_outg_columns_equal(
            rcpo_df_1=ede_df_1, 
            rcpo_df_2=ede_df_2, 
            same_order=True, 
            inplace=True, 
            cols_to_init_with_empty_lists=None
        )
        #-----
        ede_df_1 = ede_df_1.sort_values(by=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], ignore_index=True)
        ede_df_2 = ede_df_2.sort_values(by=['serialnumber', 'aep_premise_nb', 'aep_event_dt'], ignore_index=True)
        #-----
        diffs_df = Utilities_df.get_dfs_diff(ede_df_1, ede_df_2)
        #-----
        assert(save_name not in diffs_dict.keys())
        diffs_dict[save_name] = dict(
            cols_in_2_not_1=cols_in_2_not_1, 
            diffs_df=diffs_df
        )
        #-----
        if ede_df_1.equals(ede_df_2):
            continue
        else:
            diffs_df = Utilities_df.get_dfs_diff(ede_df_1, ede_df_2)
            if diffs_df.shape[0]==0:
                continue
            else:
                print(f"NOT EQUAL: {save_name}")

In [ ]:
with open(os.path.join(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\Case_Sensitive', 'diffs_dict_2.pkl'), 'wb') as handle:
    pickle.dump(diffs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
ede_df_1 = pd.read_pickle(os.path.join(save_dir_1, save_name))
cols_to_ignore = ['trsf_pole_nb', 'xf_meter_cnt', 'events_tot', 'aep_opco']
ede_df_1 = ede_df_1.drop(columns=cols_to_ignore)
#-----
rename_cols_dict = {x[0]:x[1] for x in regex_setup_df[['pivot_id', 'regex_report_title']].values.tolist()}
ede_df_1=ede_df_1.rename(columns=rename_cols_dict)

In [ ]:
col_counts = ede_df_1.columns.value_counts()
col_counts

In [ ]:
col_counts[col_counts>1]

In [ ]:
regex_setup_df[regex_setup_df['regex_report_title']=='Primary Power Up']

In [ ]:
gt1 = regex_setup_df['regex_report_title'].value_counts()
gt1

In [ ]:
regex_setup_df[regex_setup_df['regex_report_title'].isin(gt1[gt1>1].index.tolist()[1:])].sort_values(by=['regex_report_title'])

In [ ]:
diffs_dict

In [ ]:
save_names_neq = []
save_names_eq = []
for save_name_i, dict_i in diffs_dict.items():
    if len(dict_i['cols_in_2_not_1'])>0 or dict_i['diffs_df'].shape[0]>0:
        save_names_neq.append(save_name_i)
    else:
        save_names_eq.append(save_name_i)

In [ ]:
assert(len(diffs_dict)==len(save_names_neq)+len(save_names_eq))

In [ ]:
all_cols_in_2_not_1 = []
for save_name_i in save_names_neq:
    all_cols_in_2_not_1.extend(diffs_dict[save_name_i]['cols_in_2_not_1'])
all_cols_in_2_not_1 = list(set(all_cols_in_2_not_1))

In [ ]:
natsorted(all_cols_in_2_not_1)

In [ ]:
diffs_dict[save_names_neq[0]]['diffs_df']

In [ ]:
len(save_names_neq)

In [ ]:
diffs_dict[save_names_neq[8]]['diffs_df']

In [ ]:
all_diffs_dfs = pd.DataFrame()
for save_name_i in save_names_neq:
    all_diffs_dfs = pd.concat([all_diffs_dfs, diffs_dict[save_name_i]['diffs_df']])

In [ ]:
all_diffs_dfs

In [ ]:
all_diffs_dfs.index.get_level_values(1).unique().tolist()

In [ ]:
all_diffs_dfs[all_diffs_dfs.index.get_level_values(1)=='Meter had a power outage']

In [ ]:
all_diffs_dfs[all_diffs_dfs.index.get_level_values(1)=='Meter had a power outage (Unsafe power fail)'].sum()

In [ ]:
all_diffs_dfs[all_diffs_dfs.index.get_level_values(1)=='Meter had a power outage'].sum()

In [ ]:
regex_setup_df[regex_setup_df['regex_search_pattern'].str.contains('Unsafe power fail')]

In [ ]:
regex_setup_df.head(20)

In [ ]:
regex_setup_df[regex_setup_df['regex_search_pattern'].str.contains('temperature')]

In [ ]:
regex_setup_df[regex_setup_df['regex_search_pattern'].str.contains('high temperature condition')].iloc[0]['regex_search_pattern']

In [ ]:
regex_setup_df[regex_setup_df['enddeviceeventtypeid']=='3.2.0.28'].iloc[0]['regex_search_pattern']

In [ ]:
regex_setup_df[regex_setup_df['enddeviceeventtypeid']=='3.2.0.85'].iloc[0]['regex_search_pattern']

In [ ]:
regex_setup_df[regex_setup_df['enddeviceeventtypeid']=='3.2.22.28'].iloc[0]['regex_search_pattern']

In [ ]:
regex_setup_df[regex_setup_df['enddeviceeventtypeid']=='3.35.0.28'].iloc[0]['regex_search_pattern']

In [ ]:
'((?:\\s*occurred\\s*)?(?:\\s*(?:for|on)?\\s*(?:meter)?)?\\s+(?:[0-9a-zA-Z]+)(?:\\:[0-9a-zA-Z]+)+\\.?)'

In [ ]:
'((?:\\s*occurred\\s*)?(?:\\s*(?:for|on)?\\s*(?:meter)?)?\\s+(?:[0-9a-zA-Z]+)(?:\\:[0-9a-zA-Z]+)+\\.?)'

In [ ]:
strng = r'KV2c meter event Under Voltage Diagnostic flags:Phase A Voltage'
pattern = r'(KV2c meter event.*?)\s*(Diagnostic flags)\s*\:\s*(.*?)\s*$'
repl=r'\1 (\2 = \3)'
re.sub(pattern, repl, strng)

In [ ]:
Utilities.find_and_replace_in_string(strng, pattern, repl)

In [ ]:
strng

In [ ]:
save_names_neq

In [ ]:
df = pd.read_pickle(r'U:\CloudData\dovs_and_end_events_data\regex_check\EndEvents_curated\end_events_ap_20230401_20230407.pkl')

In [ ]:
df

In [ ]:
df[df['curated_reason']=='Cleared: detected a high temperature condition. (C1219 Table 3)']

In [ ]:
df[df['curated_reason']=='Cleared: detected a high temperature condition. (C1219 Table 3)'].iloc[0]['reason']

In [ ]:
strng = r'Cleared: Meter 00:13:50:05:ff:3b:57:df detected a high temperature condition. (C1219 Table 3)'
pattern = r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)'
repl=''
re.sub(pattern, repl, strng, flags=re.IGNORECASE)

In [ ]:
strng = r'Cleared: Meter 00:13:50:05:ff:3b:57:df detected a high temperature condition. (C1219 Table 3)'
pattern = r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)'
repl=''
re.sub(pattern, repl, strng, flags=0)

In [ ]:
strng = r'Cleared: Meter 00:13:50:05:ff:3b:57:df detected a high temperature condition. (C1219 Table 3)'
pattern = r'((?:\s*occurred\s*)?(?:\s*(?:for|on)?\s*(?:m|Meter)?)?\s+(?:[0-9a-zA-Z]+)(?:\:[0-9a-zA-Z]+)+\.?)'
repl=''
re.sub(pattern, repl, strng, flags=0)

In [ ]:
    ede_df = AMIEndEvents.reduce_end_event_reasons_in_df(
        df=df[df['curated_reason']=='Cleared: detected a high temperature condition. (C1219 Table 3)'], 
        reason_col='reason', 
        edetypeid_col='enddeviceeventtypeid',  
        placement_col='curated_reason'
    )

In [ ]:
ede_df.iloc[0]['curated_reason']

In [ ]:
import AMIEndEvents
reload(AMIEndEvents)
from AMIEndEvents import AMIEndEvents

In [ ]:
regex_setup_df

In [ ]:
regex_setup_df[regex_setup_df['regex_id']=='r9']

In [ ]:
regex_setup_df[regex_setup_df['regex_id'].isin(['r9', 'r10'])]

In [ ]:
regex_setup_df[regex_setup_df['regex_id']=='r9'].iloc[0]['regex_search_pattern']

In [ ]:
strng = r'Meter had a power outage. Unsafe power fail count = 11'
pattern = r'(Meter had a power outage)\.?\s*(Unsafe power fail)\s*count\s*=\s*[0-9]+'
repl=r'\1 (\2)'
re.sub(pattern, repl, strng, flags=0)

In [ ]:
strng = r'Meter had a power outage. Unsafe power fail count = 11'
pattern = r'(Meter had a power outage)\.?\s*(Unsafe power fail)\s*count\s*=\s*[0-9]+'
repl=r'\1 (\2)'
re.sub(pattern, repl, strng, flags=0)

In [ ]:
start = time.time()
df1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\RCPO_Quick\end_events_ap_20230401_20230407.pkl')
print(time.time()-start)

In [ ]:
start = time.time()
df1 = pd.read_pickle(r'U:\CloudData\dovs_and_end_events_data\regex_check\Case_Sensitive\RCPO_Quick\end_events_ap_20230401_20230407.pkl')
print(time.time()-start)

In [ ]:
idk = pd.read_pickle(r'U:\CloudData\dovs_and_end_events_data\regex_check\Case_Sensitive\RCPO_Quick\end_events_ap_20230401_20230407.pkl')

In [ ]:
natsorted(idk.columns.tolist())

In [ ]:
regex_setup_df[regex_setup_df['enddeviceeventtypeid']=='3.12.93.219']

# Checking trsf_pole_nb and xf_meter_cnt

In [ ]:
def check_trsf_pole_nb_and_meter_cnt(
    ev_summ_df
):
    r"""
    """
    #------------------------
    trsf_pole_nbs = ev_summ_df['trsf_pole_nb'].unique().tolist()
    SNs_for_xfmrs = MeterPremise.get_SNs_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs,
        include_prem_nbs=False, 
        assert_all_trsf_pole_nbs_found=False
    )
    #------------------------
    # Add nSNs
    SNs_for_xfmrs['nSNs'] = SNs_for_xfmrs['SNs'].apply(lambda x: len(set(x)))
    #------------------------
    trsf_pole_nb_and_count_df = SNs_for_xfmrs.explode('SNs').reset_index()
    #------------------------
    comp_df = pd.merge(
        ev_summ_df[['serialnumber', 'aep_premise_nb', 'trsf_pole_nb', 'xf_meter_cnt']], 
        trsf_pole_nb_and_count_df, 
        left_on='serialnumber', 
        right_on='SNs', 
        how='left'        
    )
    #------------------------
    disagree_df = comp_df[
        (comp_df['trsf_pole_nb_x']!=comp_df['trsf_pole_nb_y']) | 
        (comp_df['xf_meter_cnt']!=comp_df['nSNs'])
    ]
    return disagree_df

In [ ]:
# ede_df_1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\events_summary_vw\end_events_ap_20230401_20230407.pkl')
# ede_df_1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\regex_check\events_summary_vw\end_events_im_20230506_20230512.pkl')

In [ ]:
opcos = [
    'ap', 
    'oh', 
    'im', 
    'pso', 
    'swp', 
]
date_range = ['2023-07-28', '2023-07-28']
ede_dfs_dict = {}
for opco in opcos:
    end_events_sql_function_kwargs=dict(
        schema_name='meter_events', 
        table_name='events_summary_vw', 
        cols_of_interest=['*'], 
        date_range=date_range, 
       opco=opco
    )
    save_name = f"end_events_{opco}_{date_range[0].replace('-','')}_{date_range[1].replace('-','')}.pkl"
    #-----
    end_events = AMIEndEvents(
        df_construct_type=DFConstructType.kRunSqlQuery, 
        contstruct_df_args = dict(conn_db=conn_aws), 
        build_sql_function=AMIEndEvents_SQL.build_sql_end_events, 
        build_sql_function_kwargs=end_events_sql_function_kwargs, 
        init_df_in_constructor=True, 
        save_args=False
    )
    ede_dfs_dict[opco] = end_events.df.copy()

In [ ]:
ede_df_1 = ede_dfs_dict['oh']

In [ ]:
ede_df_1

In [ ]:
ede_df_1['trsf_pole_nb'].nunique()

In [ ]:
start=time.time()
disagree_df = check_trsf_pole_nb_and_meter_cnt(
    ede_df_1
)
print(time.time()-start)

In [ ]:
disagree_df

In [ ]:
# All trsf_pole_nbs agree (either are equal, or one is None another is NaN)
assert(disagree_df[
    (disagree_df['trsf_pole_nb_x']!=disagree_df['trsf_pole_nb_y']) & 
    ((pd.notna(disagree_df['trsf_pole_nb_x'])) & (pd.notna(disagree_df['trsf_pole_nb_y'])))
].shape[0]==0)
actually_disagree = disagree_df[
    (~disagree_df['trsf_pole_nb_x'].isin(['NETWORK', 'PRIMARY', 'TRANSMISSION'])) &
    (pd.notna(disagree_df['trsf_pole_nb_x']))
]

In [ ]:
actually_disagree

In [ ]:
actually_disagree_dfs_dict = {}
for opco, ede_df_1 in ede_dfs_dict.items():
    if opco=='tx':
        continue
    print(opco)
    disagree_df = check_trsf_pole_nb_and_meter_cnt(
        ede_df_1
    )
    # All trsf_pole_nbs agree (either are equal, or one is None another is NaN)
    assert(disagree_df[
        (disagree_df['trsf_pole_nb_x']!=disagree_df['trsf_pole_nb_y']) & 
        ((pd.notna(disagree_df['trsf_pole_nb_x'])) & (pd.notna(disagree_df['trsf_pole_nb_y'])))
    ].shape[0]==0)
    actually_disagree = disagree_df[
        (~disagree_df['trsf_pole_nb_x'].isin(['NETWORK', 'PRIMARY', 'TRANSMISSION'])) &
        (pd.notna(disagree_df['trsf_pole_nb_x']))
    ]
    actually_disagree_dfs_dict[opco] = actually_disagree

In [ ]:
actually_disagree_dfs_dict.keys()

In [ ]:
actually_disagree_dfs_dict['oh']

In [ ]:
ede_dfs_dict['oh']

In [ ]:
actually_disagree_dfs_dict['ap']

In [ ]:
for opco, df_opco in actually_disagree_dfs_dict.items():
    print(opco)
    print(df_opco.shape[0])
    print()

In [ ]:
actually_disagree_dfs_dict